In [1]:
# start docker deamon in terminal
# !sudo docker login
# OR - becuase the jupyther with allow root
# !docker login

TODO:

when running with cloud. 
if you dont need to keep the execution logs, change the cloud settings to keep it for a short while
Check that the cache if you need to set the cache to false (especially in plot process)

In [2]:
!pwd

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac


In [3]:
# # in the command line. same env
# export GOOGLE_APPLICATION_CREDENTIALS="/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/hai-gcp-genomic-beb9207a40b8.json.json"
# gcloud auth activate-service-account eila-batch@hai-gcp-genomic.iam.gserviceaccount.com --key-file=$GOOGLE_APPLICATION_CREDENTIALS

In [4]:
!gcloud auth list
# gcloud config set account


                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [5]:
import os
# os.environ["TOWER_ACCESS_TOKEN"] = 'XXX'
os.environ["TOWER_ACCESS_TOKEN"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'
os.environ["run_seqspec_modify_atac"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'
os.environ["TOWER_WORKSPACE_ID"]='247507587970818'


In [6]:
!echo $TOWER_ACCESS_TOKEN
!echo $TOWER_WORKSPACE_ID

eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz
247507587970818


In [7]:
!pwd

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac


In [8]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/atac_barcode_and_subpool.sh

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/atac_barcode_and_subpool.sh


In [9]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/

atac_barcode_and_subpool.sh		 plot_insert_size_hist-Copy1.py
barcode_rank_functions.R		 plot_insert_size_hist.py
compute_tss_script.py			 run_tabix.sh
create_sample_filtered_fragment_file.sh  scrna_plot_atac_qc_metrics-Copy1.R
generate_barcode_metadata.sh		 scrna_plot_atac_qc_metrics.R


In [10]:
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/atac_barcode_and_subpool.sh
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/run_tabix.sh
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/create_sample_filtered_fragment_file.sh
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/compute_tss_script.py
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/barcode_rank_functions.R
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/scrna_plot_atac_qc_metrics.R
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/generate_barcode_metadata.sh
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/plot_insert_size_hist.py
!chmod +x /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/joint_cell_plotting.py










chmod: cannot access '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/joint_cell_plotting.py': No such file or directory


In [11]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/plot_insert_size_hist.py


/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin/plot_insert_size_hist.py


In [12]:
# in the command line. based on:
# https://nextflow.io/blog/2023/nextflow-with-gbatch.html
# gcloud services enable batch.googleapis.com compute.googleapis.com logging.googleapis.com
# gcloud iam service-accounts create eila-batch
# gcloud iam service-accounts list 




# gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/iam.serviceAccountUser"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/batch.jobsEditor"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/logging.viewer"

# $ gcloud projects add-iam-policy-binding terra-addd4c49 \
# --member="serviceAccount:pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com" \
# --role="roles/storage.admin"


In [13]:

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/iam.serviceAccountUser"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/batch.jobsEditor"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/logging.viewer"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \

In [14]:
# !nextflow -self-update

In [15]:
!nextflow info

  Version: 23.10.0 build 5889
  Created: 15-10-2023 15:07 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [16]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/

docker				   nf_data     nf_processes
hai-gcp-genomic-beb9207a40b8.json  nf_modules  nf_workflow


In [17]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/RNA-737K-arc-v1.txt

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/RNA-737K-arc-v1.txt


In [18]:
import os
os.getcwd()

'/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac'

In [19]:
%%writefile nextflow_local.config


manifest {
  description = 'A nexflow pipeline for fg single cell atac' 
  mainScript = 'nf_wf_fg_singlecell_atac.nf'
}
profiles {
  conda_profile {
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
    }

  conda_docker_profile {
    includeConfig 'conf/conda_docker.config'
    executor = 'local'
    }

  docker_profile {
    includeConfig 'conf/docker.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'local'
  }
  google_batch_profile {
    workDir = 'gs://eila-nextflow-bucket/work/ATAC'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
    executor = 'google-batch'
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = false
    google.batch.serviceAccountEmail='pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com'
  }
    
}
params.FASTQS_SPEC_CH = '/Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_local.csv'
params.CHROMAP_IDX='/Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/Homo_sapiens.GRCh38.dna.primary_assembly.chromap.ref.index'

params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'

params.CHROMAP_TRIM_ADAPTERS = false

Overwriting nextflow_local.config


In [20]:
# %%writefile nextflow_vm.config

# includeConfig 'conf/conda.config'


# manifest {
#   description = 'A nexflow pipeline for fg single cell atac' 
#   mainScript = 'nf_wf_fg_singlecell_atac.nf'
# }

# profiles {
#   conda_profile {
#     includeConfig 'conf/conda.config'
#     conda.enabled = true
#     executor = 'local'
#     }

#   conda_docker_profile {
#     includeConfig 'conf/conda_docker.config'
#     executor = 'local'
#     }

#   docker_profile {
#     includeConfig 'conf/docker.config'
#     docker.enabled = true
#     docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
#     executor = 'local'
#   }
#   google_batch_profile {
#     workDir = 'gs://eila-nextflow-bucket/work/ATAC'
#     wave.enabled = true
#     fusion.enabled = true
#     includeConfig 'conf/docker-google-batch.config'
#     docker.enabled = true
#     docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
#     executor = 'google-batch'
#     google.project = 'hai-gcp-genomic'
#     google.location = 'us-central1'
#     google.batch.spot = false
#     google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
#   }
    
# }

# params.FASTQS_SPEC_CH = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_vm.csv'
# params.CHROMAP_IDX='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/Homo_sapiens.GRCh38.dna.primary_assembly.chromap.ref.index'

# params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
# params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'


In [21]:
%%writefile nextflow_vm.config


manifest {
  description = 'A nexflow pipeline for fg single cell atac' 
  mainScript = 'nf_wf_fg_singlecell_atac.nf'
}

profiles {
  conda_profile {
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
    }

  docker_local {
    includeConfig 'conf/docker.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    executor = 'local'
  }
  google_batch_profile {
    workDir = 'gs://eila-nextflow-bucket/work/ATAC'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    wave.enabled = true
    fusion.enabled = true
    process.scratch = false
    executor = 'google-batch'
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = true
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }
    
}

// Define ENV_SYNAPSE_TOKEN as a parameter

params.FASTQS_SPEC_CH = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_vm.csv'
params.CHROMAP_IDX='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/Homo_sapiens.GRCh38.dna.primary_assembly.chromap.ref.index'

params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'
params.ENV_SYNAPSE_TOKEN = "nextflow secrets get SYNAPSE".execute().text.trim()

params.CHROMAP_TRIM_ADAPTERS = false
params.CHROMAP_REMOVE_PCR_DUPLICATES = false
params.CHROMAP_REMOVE_PCR_DUPLICATES_AT_CELL_LEVEL = false
params.CHROMAP_TN5_SHIFT = false
params.CHROMAP_LOW_MEM = false
params.CHROMAP_BED = false
params.CHROMAP_MAX_INSERT_SIZE = 20
params.CHROMAP_BC_ERROR_THRESHOLD = 20
params.CHROMAP_BC_PROBABILITY_THRESHOLD = 0.20
params.READ_FORMAT = 'r1:0:-1,bc:0:-1'
params.DROP_REPETITIVE_THRESHOLD = false
params.MAPQ_THRESHOLD = false
params.BARCODE_TRANSLATE = 'path'
//barcode_conversion_dict_file
params.BARCODE_CONVERSION_DICT_FILE = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/na.na'
params.SUBPOOL='  '
params.ATAC_BARCODE_AND_POOL='atac_barcode_and_subpool.sh'
params.ATAC_FRAGMENTS_CUTOFF = 4
params.RUN_TABIX_SCRIPT='run_tabix.sh'
params.ATAC_TSS_BASES_FLANK = 2000
params.ATAC_TSS_COL_WITH_STRANDS_INFO = 4
params.ATAC_TSS_SMOOTHING_WINDOW_SIZE = 20
params.ATAC_TSS_REGION_BED_FILE='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/tss_regions.bed'
params.ATAC_TSS_CALCULATION_SCRIPT='compute_tss_script.py'


params.SC_ATAC_QC_PLOT_SCRIPT='scrna_plot_atac_qc_metrics.R'
params.SC_ATAC_QC_PLOT_HELPER_SCRIPT='barcode_rank_functions.R'
params.SC_ATAC_QC_BARCODE_METADATA_FILE='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/barcode_metadata.txt'
params.SC_ATAC_QC_FRAGMENT_CUTOFF=4
params.SC_ATAC_QC_BARCODE_OUTPUT_FILE='sc_atac_qc_barcode_output_file.png'
    
params.SC_ATAC_BARCODE_METADATA_SCRIPT='generate_barcode_metadata.sh'
params.SC_ATAC_BARCODE_METADATA_FILTERED_BARCODE_STATS='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/filtered_barcode_stats.csv'
params.SC_ATAC_BARCODE_METADATA_TSS_ENRICHMENT_BARCODE_STATS='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/tss_enrichment_barcode_stats.csv'

params.SC_ATAC_GENERATE_BARCODE_RANK_PLOT_SCRIPT='plot_insert_size_hist.py'
params.SC_ATAC_GENERATE_BARCODE_RANK_PLOT_PKR='PKR123'

params.SC_JOINT_CELL_BARCODE_PLOT_SCRTIP_PY='joint_cell_plotting.py'
params.SC_JOINT_CELL_BARCODE_PLOT_SCRTIP_R='joint_cell_plotting_density.R'


Overwriting nextflow_vm.config


In [22]:
!ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/tss_enrichment_barcode_stats.csv




/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/tss_enrichment_barcode_stats.csv


In [23]:
# // Try to trim adapters on 3 prime
# params.CHROMAP_TRIM_ADAPTERS = false
# // Remove PCR duplicates
# params.CHROMAP_REMOVE_PCR_DUPLICATES = false
# // Remove PCR duplicates at cell level for single cell data
# params.CHROMAP_REMOVE_PCR_DUPLICATES_AT_CELL_LEVEL = false
# // Perform Tn5 shift
# params.CHROMAP_TN5_SHIFT = false
# // Use low memory mode
# params.CHROMAP_LOW_MEM = false
# // Output mappings in BED/BEDPE format
# params.CHROMAP_BED = false
# //INT     Max insert size, only for paired-end read mapping [1000]
# params.CHROMAP_MAX_INSERT_SIZE = 20
# // INT  Max Hamming distance allowed to correct a barcode [1]
# params.CHROMAP_BC_ERROR_THRESHOLD = 20
# // FLT Min probability to correct a barcode [0.9]
# params.CHROMAP_BC_PROBABILITY_THRESHOLD = 0.20
# // STR Format for read files and barcode files [r1:0:-1,bc:0:-1 as 10x Genomics single-end format]
# params.READ_FORMAT = 'r1:0:-1,bc:0:-1'
# params.DROP_REPETITIVE_THRESHOLD = false
# // INT  Min MAPQ in range [0, 60] for mappings to be output [30]
# params.MAPQ_THRESHOLD = false
# //FILE  Convert barcode to the specified sequences during output
# params.BARCODE_TRANSLATE = 'path'



In [24]:
# %%writefile nextflow_vm_anvil.config

# includeConfig 'conf/conda.config'


# manifest {
#   description = 'A nexflow pipeline for fg single cell atac' 
#   mainScript = 'nf_wf_fg_singlecell_atac.nf'
# }

# profiles {
#   conda_profile {
#     includeConfig 'conf/conda.config'
#     conda.enabled = true
#     executor = 'local'
#     }

#   conda_docker_profile {
#     includeConfig 'conf/conda_docker.config'
#     executor = 'local'
#     }

#   docker_profile {
#     includeConfig 'conf/docker.config'
#     docker.enabled = true
#     docker.envWhitelist=['TOWER_ACCESS_TOKEN']
#     executor = 'local'
#   }
#   google_batch_profile {
#     workDir = 'gs://fc-secure-87394b1b-e8d0-4802-9671-70bc3c8b7525/nextflow'
#     includeConfig 'conf/docker-google-batch.config'
#     docker.enabled = true
#     docker.envWhitelist=['ENV_SYNAPSE_TOKEN']
#     executor = 'google-batch'
#     google.project = 'terra-addd4c49'
#     location = 'us-central1'
#     batch.spot = false
#     batch.serviceAccountEmail='pet-26869426946125b815f6b@terra-addd4c49.iam.gserviceaccount.com'
#   }
# }

# params.FASTQS_SPEC_CH = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_vm.csv'
# params.CHROMAP_IDX='/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/Homo_sapiens.GRCh38.dna.primary_assembly.chromap.ref.index'

# params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
# params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'



In [25]:
!pwd

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac


In [26]:
# !rm -r /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/work
# !rm -r /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/work
# !gsutil rm -r gs://eila-nextflow-bucket/work/ATAC

In [27]:
# !rm .nextflow*

In [28]:
# !ls nextflow.config

In [29]:
!nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume

# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile


# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile docker_profile


N E X T F L O W  ~  version 23.10.0
Launching `nf_wf_fg_singlecell_atac.nf` [small_mayer] DSL2 - revision: c741f56046
Dec 16, 2023 4:08:52 AM com.google.auth.oauth2.DefaultCredentialsProvider warnAboutProblematicCredentials
WARN: `params.tss_bases_flank` is defined multiple times -- Assignments following the first are ignored
/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/atac_fastqs_files_vm.csv
null
[-        ] process > run_seqspec_modify_atac      -
[-        ] process > run_seqspec_print            -
[-        ] process > run_chromap_test             -
[-        ] process > run_chromap_map_to_idx       -
[-        ] process > run_bgzip                    -
[-        ] process > run_tabix                    -
[-        ] process > run_merge_logs               -
[-        ] process > run_filter_fragments         -
[-        ] process > run_tabix_filtered_fragments -
[-        ] process > run_calculate_tss_enrichment -
after run_seqspec_modify_a

In [30]:
!cat .nextflow.log

Dec-16 04:08:48.811 [main] DEBUG nextflow.cli.Launcher - $> nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume
Dec-16 04:08:49.229 [main] INFO  nextflow.cli.CmdRun - N E X T F L O W  ~  version 23.10.0
Dec-16 04:08:49.258 [main] DEBUG nextflow.plugin.PluginsFacade - Setting up plugin manager > mode=prod; embedded=false; plugins-dir=/home/eila/.nextflow/plugins; core-plugins: nf-amazon@2.1.4,nf-azure@1.3.2,nf-cloudcache@0.3.0,nf-codecommit@0.1.5,nf-console@1.0.6,nf-ga4gh@1.1.0,nf-google@1.8.3,nf-tower@1.6.3,nf-wave@1.0.0
Dec-16 04:08:49.272 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Enabled plugins: []
Dec-16 04:08:49.273 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Disabled plugins: []
Dec-16 04:08:49.278 [main] INFO  org.pf4j.DefaultPluginManager - PF4J version 3.4.1 in 'deployment' mode
Dec-16 04:08:49.296 [main] INFO  org.pf4j.AbstractPluginManager - No plugins
Dec-16 04:08:49.322 [main] DEBUG nextflow.config.

In [31]:
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -resume




# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow.config -profile google_batch_profile -resume

In [32]:
# !cat .nextflow.log 

In [33]:
# !ls /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/seqspec/specs/dogmaseq-dig/fastqs

In [34]:
# !nextflow run /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_processes/nf_prcs_print.nf

In [35]:
# !ls /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_processes/nf_prcs_print.nf

In [36]:
# !cat .command.out

In [37]:
# !nextflow run nf_wf_fg_singlecell.nf -c nextflow.config -with-docker eilalan/env_synapse:latest